In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [3]:
import src.config as config
from src.inference import (
    get_feature_store
)

from datetime import datetime, timezone, timedelta

def get_current_utc_hour_ceiled():
    """Get current UTC time ceiled to the next hour"""
    current = datetime.now(timezone.utc)
    if current.minute > 0 or current.second > 0:
        # Ceil to next hour
        current = current + timedelta(hours=1)

    # Reset minutes, seconds, and microseconds
    current = current.replace(minute=0, second=0, microsecond=0)
    return current

# Usage
current_hour = get_current_utc_hour_ceiled()
print(f"Current UTC hour (ceiled): {current_hour}")

Current UTC hour (ceiled): 2025-03-06 00:00:00+00:00


In [4]:
def fetch_predictions():
    current_date = pd.Timestamp.now(tz='Etc/UTC').ceil('h')

    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_MODEL_PREDICTION,
        version=1
    )

    query = fg.select_all()
    query = query.filter(fg.pickup_hour == current_hour)

    return query.read()

In [5]:
fs = get_feature_store()
fg = fs.get_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTION,
    version=1
)

query = fg.select_all()

2025-03-05 18:18:36,555 INFO: Initializing external client
2025-03-05 18:18:36,555 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-05 18:18:37,241 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214711


In [7]:
df= query.read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (3.32s) 


In [8]:
df

,pickup_location_id,predicted_demand,pickup_hour
0,215,1.0,2025-03-05 21:00:00+00:00
1,136,0.0,2025-03-05 21:00:00+00:00
2,43,74.0,2025-03-05 21:00:00+00:00
3,152,1.0,2025-03-05 21:00:00+00:00
4,183,0.0,2025-03-05 21:00:00+00:00
...,...,...,...
751,226,3.0,2025-03-06 00:00:00+00:00
752,190,0.0,2025-03-06 00:00:00+00:00
753,128,0.0,2025-03-06 00:00:00+00:00
754,75,37.0,2025-03-06 00:00:00+00:00


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 756 entries, 0 to 755
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype                  
---  ------              --------------  -----                  
 0   pickup_location_id  756 non-null    int32                  
 1   predicted_demand    756 non-null    float64                
 2   pickup_hour         756 non-null    datetime64[us, Etc/UTC]
dtypes: datetime64[us, Etc/UTC](1), float64(1), int32(1)
memory usage: 14.9 KB


In [13]:
from datetime import datetime, timezone, timedelta

def fetch_next_hour_predictions():
    # Get current UTC time and round up to next hour
    now = datetime.now(timezone.utc)
    next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)
    print(next_hour)
    fs = get_feature_store()  
    fg = fs.get_feature_group(  
        name=config.FEATURE_GROUP_MODEL_PREDICTION,  
        version=1  
    )  
    query = fg.select_all()
    print("Completed Select all")
    df = query.read()
    print("Got All Predictions")
    # df_hour = df[df['pickup_hour'] == next_hour]

    print(f"Current UTC time: {now}")
    print(f"Next hour: {next_hour}")
    print(f"Found {len(df)} records")
    return df

# Example usage:
predictions = fetch_next_hour_predictions()

2025-03-06 00:00:00+00:00
2025-03-05 18:31:40,276 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-05 18:31:40,280 INFO: Initializing external client
2025-03-05 18:31:40,280 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-05 18:31:40,750 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214711
Completed Select all
2025-03-05 18:32:36,757 ERROR: Flight returned timeout error, with message: Deadline Exceeded
Traceback (most recent call last):
  File "/home/ashish159565/miniconda3/envs/appliedml/lib/python3.11/site-packages/hsfs/core/arrow_flight_client.py", line 205, in __init__
    self._health_check()
  File "/home/ashish159565/miniconda3/envs/appliedml/lib/python3.11/site-packages/retrying.py", line 56, in wrapped_f
    return Retrying(*dargs, **dkw).call(f, *args, **kw)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/ashish159565/miniconda3/envs/appliedml/lib/pyt

2025-03-05 18:32:41,764 ERROR: Flight returned timeout error, with message: Deadline Exceeded
Traceback (most recent call last):
  File "/home/ashish159565/miniconda3/envs/appliedml/lib/python3.11/site-packages/hsfs/core/arrow_flight_client.py", line 364, in afs_error_handler_wrapper
    return func(instance, *args, **kw)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/ashish159565/miniconda3/envs/appliedml/lib/python3.11/site-packages/hsfs/core/arrow_flight_client.py", line 427, in read_query
    return self._get_dataset(
           ^^^^^^^^^^^^^^^^^^
  File "/home/ashish159565/miniconda3/envs/appliedml/lib/python3.11/site-packages/retrying.py", line 56, in wrapped_f
    return Retrying(*dargs, **dkw).call(f, *args, **kw)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/ashish159565/miniconda3/envs/appliedml/lib/python3.11/site-packages/retrying.py", line 257, in call
    return attempt.get(self._wrap_exception)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  

FeatureStoreException: Could not read data using Hopsworks Feature Query Service.

In [26]:
now = datetime.now(timezone.utc)

In [27]:
predictions

,pickup_location_id,predicted_demand,pickup_hour


In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252 entries, 0 to 251
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype                  
---  ------              --------------  -----                  
 0   pickup_location_id  252 non-null    int32                  
 1   predicted_demand    252 non-null    float64                
 2   pickup_hour         252 non-null    datetime64[us, Etc/UTC]
dtypes: datetime64[us, Etc/UTC](1), float64(1), int32(1)
memory usage: 5.1 KB


In [29]:
import pandas as pd
current_date = pd.Timestamp.now(tz='Etc/UTC').ceil('h')

In [30]:
dt = current_date.ceil('h')


In [14]:
fs = get_feature_store()  
fg = fs.get_feature_group(  
    name=config.FEATURE_GROUP_MODEL_PREDICTION,  
    version=1  
)  
# query = fg.select_all() 
# query = query.filter(fg.pickup_hour >= "2025-02-13 04:00")
# query = query.filter(fg.pickup_hour > "2025-02-13 05:00:00")  
results = fg.read()

2025-03-05 18:33:34,124 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-05 18:33:34,137 INFO: Initializing external client
2025-03-05 18:33:34,138 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-05 18:33:34,715 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214711
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.34s) 


In [22]:
results.to_csv("test.csv")

In [ ]:
results[results["pickup_hour"] == 00]

,pickup_location_id,predicted_demand,pickup_hour


In [124]:
fg.filter((fg.pickup_hour >= "2025-02-13") & (fg.pickup_hour < "2025-02-14")).read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.33s) 


,pickup_location_id,predicted_demand,pickup_hour
0,22,0.0,2025-02-13 04:00:00+00:00
1,131,0.0,2025-02-13 04:00:00+00:00
2,36,0.0,2025-02-13 04:00:00+00:00
3,83,0.0,2025-02-13 04:00:00+00:00
4,152,1.0,2025-02-13 04:00:00+00:00
...,...,...,...
251,173,0.0,2025-02-13 04:00:00+00:00
252,229,61.0,2025-02-13 04:00:00+00:00
253,208,0.0,2025-02-13 04:00:00+00:00
254,45,4.0,2025-02-13 04:00:00+00:00


In [108]:
results

,pickup_location_id,predicted_demand,pickup_hour
0,22,0.0,2025-02-13 04:00:00+00:00
1,131,0.0,2025-02-13 04:00:00+00:00
2,36,0.0,2025-02-13 04:00:00+00:00
3,83,0.0,2025-02-13 04:00:00+00:00
4,152,1.0,2025-02-13 04:00:00+00:00
...,...,...,...
251,173,0.0,2025-02-13 04:00:00+00:00
252,229,61.0,2025-02-13 04:00:00+00:00
253,208,0.0,2025-02-13 04:00:00+00:00
254,45,4.0,2025-02-13 04:00:00+00:00


In [161]:
def fetch_next_hour_predictions():
    # Get current UTC time and round up to next hour
    now = datetime.now(timezone.utc)
    next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)

    fs = get_feature_store()
    fg = fs.get_feature_group(name=config.FEATURE_GROUP_MODEL_PREDICTION, version=1)

    # Then filter for next hour in the DataFrame
    df_hour = df[df["pickup_hour"] == next_hour]

    print(f"Current UTC time: {now}")
    print(f"Next hour: {next_hour}")
    print(f"Found {len(df_hour)} records")
    return df_hour

In [162]:
predictions = fetch_next_hour_predictions()

2025-02-13 17:56:01,671 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-02-13 17:56:01,681 INFO: Initializing external client
2025-02-13 17:56:01,684 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-02-13 17:56:03,247 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1211551
Current UTC time: 2025-02-13 22:56:01.671315+00:00
Next hour: 2025-02-13 23:00:00+00:00
Found 256 records


In [155]:
fs = get_feature_store()
fg = fs.get_feature_group(name=config.FEATURE_GROUP_MODEL_PREDICTION, version=1)



2025-02-13 17:53:22,384 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-02-13 17:53:22,397 INFO: Initializing external client
2025-02-13 17:53:22,398 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-02-13 17:53:23,798 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1211551


In [156]:
df = fg.read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.84s) 


In [158]:
df["pickup_hour"].max()

Timestamp('2025-02-13 23:00:00+0000', tz='Etc/UTC')

In [159]:
now = datetime.now(timezone.utc)
next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)
print(next_hour)

2025-02-13 23:00:00+00:00


In [160]:
df[df["pickup_hour"] == next_hour]

,pickup_location_id,predicted_demand,pickup_hour
34594,211,29.0,2025-02-13 23:00:00+00:00
34595,53,0.0,2025-02-13 23:00:00+00:00
34596,112,0.0,2025-02-13 23:00:00+00:00
34598,6,0.0,2025-02-13 23:00:00+00:00
34599,240,0.0,2025-02-13 23:00:00+00:00
...,...,...,...
34905,153,0.0,2025-02-13 23:00:00+00:00
34906,187,0.0,2025-02-13 23:00:00+00:00
34907,237,272.0,2025-02-13 23:00:00+00:00
34908,251,0.0,2025-02-13 23:00:00+00:00
